## Data Handling

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
np.random.seed(4444)

In [2]:
df = pd.read_csv('../../../../../../jaeyeun/01_nh_poc/17_add_prep_text_to_excel/split_70_15_15_prep_v9.csv')

df.head()

,file_name,label,raw_text,dataset_n,train_val_test,prep_v9_text
0,R1509261.txt,0,보 도 자 료\nhttp://www.msip.go.kr 보도일시 2015. 9. 4...,0,0,온라인N 제도N 혁신N 관호N 미래N 사이트N 플러그인N 지원N 대응N 현황N 공개...
1,R2003733.txt,0,보도일시 2020. 3. 18.(수) 조간(온라인 3. 17. 12:00)부터 보도...,0,0,온라인N 정보N 산업N 기반N 유승N ictS 분야N 창업N 벤처N 지원N 사업N ...
2,D1507076-1.txt,0,보 도 자 료\nhttp://www.msip.go.kr 보도일시 2015. 7. 1...,0,0,이후N 연구N 예산N 내년도N 연구N 개발N r&dN 경제N 혁신N 미래N 성장N ...
3,R2005031.txt,0,보 도 자 료\n배포일시 2020. 4. 29.(수) 총 4매(본문2) 담당 부서 ...,0,0,첨단N 항공N 과장N 문석N 홍일산N 이후N 가능N 일상N 시대N 개막N 시행N 전...
4,R2006226.txt,0,<전매체> 2020년 6월 3일(수) 10:00(국무회의 개최시)부터 보도하여 주시...,0,0,국무회의N 개최N 문의N 기획N 재정관N 과장N 서기관N 위기N 혁신N 기회N 벤처...


* exp3 combination

In [3]:
X_train_df = df[df['train_val_test'] == 0]
X_val_df = df[df['train_val_test'] == 1]
X_test_df = df[df['train_val_test'] == 2]
y_train = df[df['train_val_test'] == 0]['label']
y_val = df[df['train_val_test'] == 1]['label']
y_test = df[df['train_val_test'] == 2]['label']

In [4]:
print(X_train_df.shape)
print(X_val_df.shape)
print(X_test_df.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(7740, 6)
(1662, 6)
(1671, 6)
(7740,)
(1662,)
(1671,)


In [5]:
n_features = 100000
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.9, # 0.9 만큼의 문서 이상에서 나오면 거른다.
    min_df=5, # 5개 미만의 문서에서 나오면 거른다.
    sublinear_tf = True, # tf value를 완만하게 처리 (outlier 처리 효과)
    ngram_range = (1, 3),
    max_features=n_features)

* data is 'prep_v9_text'

In [6]:
X_train_tfidf = tfidf_vectorizer.fit(X_train_df['prep_v9_text'])
# X_test_tfidf = tfidf_vectorizer.fit_transform(prep_text_test)
# X_test_hash = hash_vectorizer.fit_transform(prep_text_test)

In [7]:
X_train = X_train_tfidf.transform(X_train_df['prep_v9_text'])
X_val = X_train_tfidf.transform(X_val_df['prep_v9_text'])
X_test = X_train_tfidf.transform(X_test_df['prep_v9_text'])

In [8]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(7740, 100000)
(1662, 100000)
(1671, 100000)


## Classify

* LightGBM RandomSearch

In [10]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgbm

In [12]:
lgbm_clf = lgbm.LGBMClassifier()

lgbm_param_grid = {'learning_rate': [.01, .015, .025, .05, .1],
#                   'Gamma': [.05, .1, .3, .5, .7, .9, 1],
                  'max_depth': [3, 5, 7, 9, 12, 15, 17, 25],
                  'min_child_weight': [1, 3, 5, 7],
                  'subsample': np.linspace(0.6, 1, 5)}

fit_params = {"early_stopping_rounds" : 20,
             "eval_metric" : "multi_error",
             "eval_set" : [(X_train, y_train), (X_val, y_val)]}

# Create a random search object
lgbm_random = RandomizedSearchCV(estimator = lgbm_clf,
                                param_distributions = lgbm_param_grid,
                                n_iter = 20, # n_iters in param combinations
                                scoring='accuracy',
                                n_jobs=-1,
                                cv = 5,
                                refit=True,
                                return_train_score = True,
                                verbose=10)

# Fit to the training data
lgbm_random.fit(X_train, y_train, **fit_params)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 34.0min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 49.1min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 56.8min
[Parallel(n_jobs=-1)]: Done  48 out of 100 | elapsed: 86.6min remaining: 93.8min
[Parallel(n_jobs=-1)]: Done  59 out of 100 | elapsed: 96.8min remaining: 67.2min
[Parallel(n_jobs=-1)]: Done  70 out of 100 | elapsed: 107.8min remaining: 46.2min
[Parallel(n_jobs=-1)]: Done  81 out of 100 | elapsed: 122.6min remaining: 28.8min
[Parallel(n_jobs=-1)]: Done  92 out of 100 | elapsed: 135.6min remaining: 11.8min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 154.3min finished


[1]	training's multi_error: 0.450388	training's multi_logloss: 1.95442	valid_1's multi_error: 0.477738	valid_1's multi_logloss: 1.97648
Training until validation scores don't improve for 20 rounds
[2]	training's multi_error: 0.247416	training's multi_logloss: 1.74089	valid_1's multi_error: 0.309868	valid_1's multi_logloss: 1.79133
[3]	training's multi_error: 0.187597	training's multi_logloss: 1.57555	valid_1's multi_error: 0.262936	valid_1's multi_logloss: 1.64488
[4]	training's multi_error: 0.152455	training's multi_logloss: 1.43856	valid_1's multi_error: 0.240072	valid_1's multi_logloss: 1.5269
[5]	training's multi_error: 0.139147	training's multi_logloss: 1.3231	valid_1's multi_error: 0.229242	valid_1's multi_logloss: 1.42901
[6]	training's multi_error: 0.129328	training's multi_logloss: 1.22368	valid_1's multi_error: 0.222022	valid_1's multi_logloss: 1.34512
[7]	training's multi_error: 0.121835	training's multi_logloss: 1.13767	valid_1's multi_error: 0.219615	valid_1's multi_loglos

RandomizedSearchCV(cv=5, estimator=LGBMClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.015, 0.025,
                                                          0.05, 0.1],
                                        'max_depth': [3, 5, 7, 9, 12, 15, 17,
                                                      25],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'subsample': array([0.6, 0.7, 0.8, 0.9, 1. ])},
                   return_train_score=True, scoring='accuracy', verbose=10)

In [13]:
pred_train = lgbm_random.best_estimator_.predict(X_train)
pred_val = lgbm_random.best_estimator_.predict(X_val)
pred_test = lgbm_random.best_estimator_.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_val, pred_val))
print(accuracy_score(y_test, pred_test))

0.99328165374677
0.8273164861612515
0.812687013764213


In [15]:
cv_result_df = pd.DataFrame(lgbm_random.cv_results_)

df_list = []
for i in range(20):
    df_list.append(pd.DataFrame([cv_result_df.loc[i, "params"]]))
    
param_table = pd.concat(df_list)

param_table['mean_test_score'] = cv_result_df['mean_test_score'].values

param_table.sort_values(by='mean_test_score', axis=0)

,subsample,min_child_weight,max_depth,learning_rate,mean_test_score
0,0.7,7,3,0.010,0.730620
0,0.9,3,3,0.025,0.759302
0,0.7,3,3,0.025,0.759302
0,0.9,7,7,0.010,0.771447
0,1.0,7,17,0.010,0.771964
0,0.8,7,9,0.015,0.779587
0,0.7,1,15,0.015,0.783075
0,1.0,1,15,0.015,0.783075
0,0.9,1,9,0.015,0.784755
0,0.7,7,17,0.025,0.789018
